# VariantSpark-k Demo HipsterIndex

Demo HipsterIndex job - takes < 3+ minutes to run w/ 4 qty of r4.rxlarge EC2 instance & Kubernetes

### Parameters
 - input bucket location - CSIRO public S3 bucket (Sydney)
 - input files `hipster...` and fc `label`
 - number of Spark executors 20 w/ 1 GM RAM (default) each
 - number of RF trees 500, w/ batch of 16
 - default `mtry`
 - calculate OOB as `-ro` paramater is configured

In [1]:
%%bash

set -e

MASTER=https://kubernetes.default.svc:443
INPUT_BUCKET=variant-spark

function fatal_error () {
	echo "ERROR: $1" 1>&2
	exit 1
}

if [ -z ${MASTER+x} ];
    then
        echo "You must set the MASTER environment variable to a kubernetes API endpoint";
        echo "Example: https://ABC.sk1.us-west-2.eks.amazonaws.com:443"
        exit 1
fi

if [ -z ${INPUT_BUCKET+x} ];
    then
        echo "You must set the INPUT_BUCKET environment variable to a bucket containing input data";
        echo "Example: variant-spark-k-storage"
        exit 1
fi

[[ $(type -P "spark-submit") ]] || fatal_error  "\`spark-submit\` cannot be found. Please make sure it's on your PATH."

spark-submit \
    --class au.csiro.variantspark.cli.VariantSparkApp \
    --driver-class-path ./conf \
    --master k8s://${MASTER} \
    --deploy-mode cluster \
    --name VariantSparkHipsterSmall \
    --conf spark.kubernetes.authenticate.driver.serviceAccountName=spark \
    --conf spark.executor.instances=20 \
    --conf spark.kubernetes.container.image=jamesrcounts/variantspark:002 \
    --jars http://central.maven.org/maven2/org/apache/hadoop/hadoop-aws/2.7.3/hadoop-aws-2.7.3.jar,http://central.maven.org/maven2/com/amazonaws/aws-java-sdk/1.7.4/aws-java-sdk-1.7.4.jar,http://central.maven.org/maven2/joda-time/joda-time/2.9.9/joda-time-2.9.9.jar \
    local:///opt/spark/jars/variant-spark_2.11-0.2.0-SNAPSHOT-all.jar importance \
        -if s3a://${INPUT_BUCKET}/datasets/hipsterIndex/hipster.vcf \
        -ff s3a://${INPUT_BUCKET}/datasets/hipsterIndex/hipster_labels.txt \
        -fc label \
        -v \
        -rn 500 \
        -rbs 16 \
        -ro "$@"


2018-08-05 02:58:51 WARN  DriverServiceBootstrapStep:66 - Driver's hostname would preferably be variantsparkhipstersmall-9a0cd5de07f9359ab1af37a5a6a1ac30-driver-svc, but this is too long (must be <= 63 characters). Falling back to use spark-1533437931792-driver-svc as the driver service's name.
2018-08-05 02:58:52 INFO  LoggingPodStatusWatcherImpl:54 - State changed, new state: 
	 pod name: variantsparkhipstersmall-9a0cd5de07f9359ab1af37a5a6a1ac30-driver
	 namespace: default
	 labels: spark-app-selector -> spark-573712c44b084e528bb4c2f2bcc244b9, spark-role -> driver
	 pod uid: 7c0f05c5-985b-11e8-b501-0af7ed887422
	 creation time: 2018-08-05T02:58:52Z
	 service account name: spark
	 volumes: spark-init-properties, download-jars-volume, download-files-volume, spark-token-cv25b
	 node name: N/A
	 start time: N/A
	 container images: N/A
	 phase: Pending
	 status: []
2018-08-05 02:58:52 INFO  LoggingPodStatusWatcherImpl:54 - State changed, new state: 
	 pod name: variantsparkhipstersmall-9a